In [ ]:
!pip3 install Keras-Preprocessing

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import categorical_accuracy
from sklearn.metrics import hamming_loss
from sklearn.metrics import classification_report

In [ ]:
data = pd.read_csv("data_preprocess.csv")

In [ ]:
data

,Unnamed: 0,Summary,Safety Issues,Location,Title,Date Occurred,Decision Events,Skill Based Events,Routine Violation,Physical Environment,Technical Environment,Communications,Awareness Cognitive Factors,Inadequate Supervision,Planned Inappropriate Operations,Organizational Processes,Resource Management,Overall Info
0,0,a shore worker was fatally injured when he bec...,the deceased used an unapproved means of acces...,"Ilfracombe Harbour, England",Accident to shore worker while disembarking pa...,3 August 2015,0,1,0,0,0,0,0,1,0,1,0,Summary: a shore worker was fatally injured wh...
1,1,kuzma minin grounded after dragging its anchor...,the financial situation of the murmansk shippi...,"Falmouth Bay, England",Grounding of bulk carrier Kuzma Minin,18 December 2018,0,0,0,1,0,0,0,0,0,0,1,Summary: kuzma minin grounded after dragging i...
2,2,"at 2100 bst on 9 august 2006, the antigua and ...",the master was aware that an appropriate chart...,"Mostyn Bank, Wales",Dragging anchor and grounding of general cargo...,10 August 2006,0,1,0,0,0,1,0,0,1,1,0,"Summary: at 2100 bst on 9 august 2006, the ant..."
3,3,"on june 22, 2020, about 0230 local time, the d...",the probable cause of the collapse of containe...,"Hilo, HI",Container Damage and Loss aboard Deck Cargo Ba...,6/22/2020,0,0,0,0,0,0,0,0,0,1,1,"Summary: on june 22, 2020, about 0230 local ti..."
4,4,"the yacht ouzo sailed from bembridge, isle of ...",ouzo was not seen or otherwise detected by the...,"South of the Isle of Wight, England",Sinking of sailing yacht Ouzo after encounter ...,20 August 2006,1,0,0,0,0,0,0,0,0,1,0,"Summary: the yacht ouzo sailed from bembridge,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,704,"about 1725 on august 24, a fire was detected i...",the probable cause of the fire aboard fishing ...,"Newburyport, MA",Fire and Subsequent Sinking of Commercial Fish...,8/24/2018,1,0,0,0,1,0,0,0,0,0,1,"Summary: about 1725 on august 24, a fire was d..."
705,705,"on 3 june 1993 at 0543, the british trent and ...",failure to comply with the international regul...,"Near Akkaert Bank, Belgium",Collision between oil tanker British Trent and...,3 June 1993,1,1,1,1,0,0,1,0,1,1,0,"Summary: on 3 june 1993 at 0543, the british t..."
706,706,"on 10 june 2000, the mariama k (a 29.76m long ...",mariama k’s bilge pumps were defective on depa...,"Douarnenez, France",Carbon monoxide poisoning on stern trawler Mar...,10 June 2000,0,0,0,0,1,0,0,0,0,0,1,"Summary: on 10 june 2000, the mariama k (a 29...."
707,707,"at 0453 utc on 9 april 2011, the gibraltar reg...",philipp’s oow had not properly assessed the ri...,6nm south of the Isle of Man,Collision between container vessel Philipp and...,9 April 2011,1,0,1,0,0,1,0,0,0,0,0,"Summary: at 0453 utc on 9 april 2011, the gibr..."


In [ ]:
x = data["Overall Info"].values.tolist()
y = data[data.columns[6:17]].values.tolist()

In [ ]:
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
word_index = tokenizer.word_index
data1 = pad_sequences(sequences, maxlen=1000)

In [ ]:
y = np.array(y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data1, y, test_size=0.15, random_state=101)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=101)

In [ ]:
clf = Sequential()
clf.add(Dense(32, activation='relu', input_dim=1000))
# clf.add(Dense(16, activation='relu'))
clf.add(Dense(11, activation='sigmoid'))
clf.compile(loss='binary_crossentropy', optimizer=SGD(), metrics=[categorical_accuracy])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = clf.fit(x_train, y_train, epochs=1, batch_size=512, verbose=0)

In [ ]:
y_pred = clf.predict(x_test)

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [ ]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 0]])

In [ ]:
for i in range(len(y_pred)):
  for j in range(len(y_pred[0])):
    y_pred[i][j] = y_pred[i][j] > 0.5

In [ ]:
y_pred

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
loss = hamming_loss(y_test, y_pred)

In [ ]:
loss

0.3959218351741716

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.56      0.40        36
           1       0.00      0.00      0.00        30
           2       0.00      0.00      0.00        24
           3       0.24      0.45      0.32        22
           4       0.50      1.00      0.67        53
           5       0.15      0.38      0.21        21
           6       0.00      0.00      0.00        28
           7       0.00      0.00      0.00        24
           8       0.33      0.05      0.08        21
           9       0.00      0.00      0.00        37
          10       0.25      1.00      0.40        27

   micro avg       0.31      0.37      0.34       323
   macro avg       0.16      0.31      0.19       323
weighted avg       0.19      0.37      0.23       323
 samples avg       0.31      0.42      0.33       323



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
